In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg_graph import (
    HeisenbergModel,
    get_n_steps,
    get_graph,
    get_positions,
    get_initial_layout,
    get_prob0,
    extract_probs,
)
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import mthree
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
# Option1: Use IBM Quantum backend.
# Set up the Qiskit Runtime service (this is a one-time setup)
# QiskitRuntimeService.save_account(
#     token="YOUR_API_TOKEN",
#     channel="ibm_quantum",
# )

# Load saved credentials
service = QiskitRuntimeService()
# backend_qpu = service.least_busy(simulator=False, interactional=True)
backend_qpu = service.backend("ibm_marrakesh")

# Option2: Use local AerSimulator as the backend.
backend_sim = AerSimulator()

noise_model = NoiseModel.from_backend(backend_qpu)
backend_sim_noisy = AerSimulator(noise_model=noise_model)

print(f"Using backend QPU: {backend_qpu}")
print(f"Using backend simulator: {backend_sim}")
print(f"Using backend noisy simulator: {backend_sim_noisy}")

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend simulator: AerSimulator('aer_simulator')
Using backend noisy simulator: AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['x', 'measure', 'reset', 'cz', 'id', 'sx']>)


In [5]:
# Only for CP1
%cd fourier_learning_ibm/

/home/jovyan/fourier_learning_ibm


In [6]:
# Load parameters
# current_time can be found in data_generate.ipynb

CURRENT_TIME = "2024-12-11T00:57Z"

path = f"./data/{CURRENT_TIME}"

# Load the graphs as a binary file
with open(f"{path}/params_object.pkl", "rb") as f:
    params_object = pickle.load(f)

graphs = params_object["graphs"]
all_Js = params_object["Js"]
all_expected_values = params_object["expected_values"]

# Load the parameters as a JSON file
with open(f"{path}/params_text.json", "r") as f:
    params = json.load(f)

n_samples = params["n_samples"]
n_qubits = params["n_qubits"]
graph_type = params["graph_type"]
backend_qpu_name = params["backend_qpu_name"]
beta = params["beta"]
C = params["C"]
n_features = params["n_features"]
times = params["times"]

for key, value in params.items():
    print(f"{key}: {value}")

n_samples: 56
n_qubits: 6
graph_type: line
backend_qpu_name: ibm_marrakesh
beta: 1
C: 7.414563484121308
n_features: 10
times: [0.0, 0.42370567873856435, 0.8474113574771287, 1.271117036215693, 1.6948227149542574, 2.118528393692822, 2.542234072431386, 2.9659397511699503, 3.389645429908515, 3.8133511086470793]


# Create Fourier feature

## Trotter simulation (Exact)

In [7]:
# Exact simulation
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# Generate the dataset with Fourier features
for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    heisenberg_exact = HeisenbergModel(n_qubits, graphs[i])
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        prob_phase0 = heisenberg_exact.exact_simulation(times[k], phase=0)
        prob_phase1 = heisenberg_exact.exact_simulation(times[k], phase=1)
        prob_phase2 = heisenberg_exact.exact_simulation(times[k], phase=2)
        prob_phase3 = heisenberg_exact.exact_simulation(times[k], phase=3)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_exact_df = pd.DataFrame(data, columns=columns)
display(data_exact_df)

# Save the exact data
data_exact_df.to_json(f"{path}/data_exact_df.json")
with open(f"{path}/probs_phase0_exact.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1_exact.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2_exact.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3_exact.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/56


/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


Calculating features for sample 1/56
Calculating features for sample 2/56
Calculating features for sample 3/56
Calculating features for sample 4/56
Calculating features for sample 5/56
Calculating features for sample 6/56
Calculating features for sample 7/56
Calculating features for sample 8/56
Calculating features for sample 9/56
Calculating features for sample 10/56
Calculating features for sample 11/56
Calculating features for sample 12/56
Calculating features for sample 13/56
Calculating features for sample 14/56
Calculating features for sample 15/56
Calculating features for sample 16/56
Calculating features for sample 17/56
Calculating features for sample 18/56
Calculating features for sample 19/56
Calculating features for sample 20/56
Calculating features for sample 21/56
Calculating features for sample 22/56
Calculating features for sample 23/56
Calculating features for sample 24/56
Calculating features for sample 25/56
Calculating features for sample 26/56
Calculating features 

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,1.0,0.745704,0.403873,0.258853,0.429043,-0.041195,0.139711,-0.046187,-0.170563,...,-0.271394,0.205053,-0.133522,0.124067,0.011621,0.031929,0.004498,0.043264,-0.011692,7.391106
1,1,1.0,0.521817,-0.235798,-0.075684,-0.069862,0.019650,0.237055,0.238121,0.212837,...,0.089689,-0.157878,0.134332,-0.143243,-0.014219,-0.085757,-0.212713,-0.122219,0.079479,7.380861
2,2,1.0,0.802638,-0.235200,0.344098,-0.277211,-0.103369,-0.064801,-0.332487,0.258295,...,0.471551,-0.117526,0.476092,0.015523,0.322248,0.027281,0.131723,-0.029976,0.022876,1.560675
3,3,1.0,0.739680,0.213438,0.270651,0.214191,0.040426,0.055534,0.045735,0.061322,...,0.352569,-0.218908,0.603565,-0.464874,0.524557,-0.470929,0.236146,-0.246830,0.027375,5.663484
4,4,1.0,0.710060,-0.189786,0.274111,-0.087356,0.006286,0.017487,-0.197054,0.179128,...,0.469646,0.058647,0.583356,0.225884,0.450033,0.224078,0.207961,0.037204,0.035246,1.889904
5,5,1.0,0.892642,-0.129649,0.672960,-0.261482,0.509202,-0.329780,0.429986,-0.284286,...,-0.182673,0.219559,-0.108396,0.110986,-0.037749,0.067931,0.109376,0.081043,0.306286,2.056023
6,6,1.0,0.751604,-0.218926,0.352644,-0.170171,0.102798,-0.115589,-0.125449,-0.079136,...,0.138792,0.133927,0.248563,0.389739,0.059865,0.435972,-0.079595,0.521116,-0.119648,1.492621
7,7,1.0,0.544153,-0.778113,-0.342619,-0.774325,-0.748485,-0.050095,-0.321015,0.580746,...,0.463383,0.575093,-0.198597,0.073687,-0.625062,-0.511875,-0.358219,-0.531593,0.258712,0.238580
8,8,1.0,0.852975,-0.075896,0.642399,-0.036575,0.579209,-0.043304,0.474411,-0.163519,...,-0.209838,0.044887,-0.150126,-0.040117,-0.160883,-0.149370,-0.247730,-0.241151,-0.269764,1.960940
9,9,1.0,0.710604,0.200737,0.196163,0.228901,-0.029231,0.065537,-0.018678,-0.152263,...,-0.296061,-0.340625,-0.373555,-0.286103,-0.432689,0.041722,-0.407902,0.229605,-0.205112,5.800941


## Trotter simulation (Simulator, Noise free)

In [8]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    heisenberg_sim = HeisenbergModel(n_qubits, graphs[i], backend=backend_sim)

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=0)
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=1)
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=2)
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=3)
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/56
Creating circuits for sample 1/56
Creating circuits for sample 2/56
Creating circuits for sample 3/56
Creating circuits for sample 4/56
Creating circuits for sample 5/56
Creating circuits for sample 6/56
Creating circuits for sample 7/56
Creating circuits for sample 8/56
Creating circuits for sample 9/56
Creating circuits for sample 10/56
Creating circuits for sample 11/56
Creating circuits for sample 12/56
Creating circuits for sample 13/56
Creating circuits for sample 14/56
Creating circuits for sample 15/56
Creating circuits for sample 16/56
Creating circuits for sample 17/56
Creating circuits for sample 18/56
Creating circuits for sample 19/56
Creating circuits for sample 20/56
Creating circuits for sample 21/56
Creating circuits for sample 22/56
Creating circuits for sample 23/56
Creating circuits for sample 24/56
Creating circuits for sample 25/56
Creating circuits for sample 26/56
Creating circuits for sample 27/56
Creating circuits for sample 2

In [9]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/56
Running circuits for sample 1/56
Running circuits for sample 2/56
Running circuits for sample 3/56
Running circuits for sample 4/56
Running circuits for sample 5/56
Running circuits for sample 6/56
Running circuits for sample 7/56
Running circuits for sample 8/56
Running circuits for sample 9/56
Running circuits for sample 10/56
Running circuits for sample 11/56
Running circuits for sample 12/56
Running circuits for sample 13/56
Running circuits for sample 14/56
Running circuits for sample 15/56
Running circuits for sample 16/56
Running circuits for sample 17/56
Running circuits for sample 18/56
Running circuits for sample 19/56
Running circuits for sample 20/56
Running circuits for sample 21/56
Running circuits for sample 22/56
Running circuits for sample 23/56
Running circuits for sample 24/56
Running circuits for sample 25/56
Running circuits for sample 26/56
Running circuits for sample 27/56
Running circuits for sample 28/56
Running circuits for sam

In [10]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,8cc15640-a6fc-46db-a053-b604da074734,0,DONE
1,475aebe9-eff5-43b5-a4e8-06b165441062,1,DONE
2,f484e96a-d6d8-4e3b-a130-0de6ee2fb03a,2,DONE
3,765ec9f8-e68d-4ca7-b7b1-d87cda3a8ace,3,DONE
4,722fc79c-78c2-4f22-8f65-f461919c2f1d,4,DONE
5,aa68b787-8ff3-4c3e-9373-39b9bfa06c48,5,DONE
6,828cf0a4-fc80-4c8a-9199-d6f4b195e1e2,6,DONE
7,44cffe74-1a23-47d5-ab6b-c9d2b933fb00,7,DONE
8,08c1b61b-f054-4879-be82-a8084cef7e16,8,DONE
9,6953b303-27ed-40c0-8c4b-f4a0626d9eb5,9,DONE


In [11]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_df = pd.DataFrame(data_sim, columns=columns)
display(data_sim_df)

# Save the simulation data
data_sim_df.to_json(f"{path}/data_sim_df.json")
with open(f"{path}/probs_phase0_sim.json", "w") as f:
    json.dump(probs_phase0_sim, f)

with open(f"{path}/probs_phase1_sim.json", "w") as f:
    json.dump(probs_phase1_sim, f)

with open(f"{path}/probs_phase2_sim.json", "w") as f:
    json.dump(probs_phase2_sim, f)

with open(f"{path}/probs_phase3_sim.json", "w") as f:
    json.dump(probs_phase3_sim, f)

 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No co

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,1.0,0.709525,0.385296,0.229803,0.419956,-0.031027,0.123494,-0.041531,-0.157682,...,-0.262867,0.162331,-0.118811,0.021603,0.059439,0.005170,0.102218,-0.016330,-0.001151,7.391106
1,1,1.0,0.540279,-0.230309,-0.015290,-0.099996,0.041851,0.236345,0.217299,0.404456,...,0.097131,-0.172826,0.004039,-0.222469,-0.091116,0.062821,0.092489,0.019352,0.142241,7.380861
2,2,1.0,0.799777,-0.217869,0.376729,-0.194887,-0.090183,-0.060270,-0.323246,0.233479,...,0.461524,-0.103002,0.459561,0.044115,0.324562,0.043992,0.183460,-0.035804,0.099785,1.560675
3,3,1.0,0.746355,0.149259,0.231822,0.151833,0.040060,0.044988,0.003601,-0.026894,...,0.256040,0.062236,0.548511,0.182312,0.578011,-0.294067,0.445342,-0.185077,0.236884,5.663484
4,4,1.0,0.710030,-0.209491,0.188055,-0.099737,-0.006476,0.011961,-0.014781,0.047551,...,0.400116,0.360221,0.431313,0.475521,0.201246,0.291627,-0.029284,0.117464,-0.010338,1.889904
5,5,1.0,0.890789,-0.121721,0.729545,-0.165242,0.535940,-0.204641,0.399837,-0.056306,...,-0.045373,0.237126,0.087744,0.078703,0.230655,0.008592,0.242338,-0.164835,0.311023,2.056023
6,6,1.0,0.790701,-0.183870,0.354028,-0.233618,0.051827,-0.165822,-0.072636,0.021344,...,0.139365,0.294590,0.222783,0.594444,-0.115978,0.604992,-0.403043,0.393843,-0.652164,1.492621
7,7,1.0,0.562660,-0.738305,-0.238959,-0.769724,-0.736536,-0.098691,-0.398999,0.483599,...,0.537017,0.554365,0.024531,0.331702,-0.383031,-0.222430,-0.507149,-0.436467,-0.088664,0.238580
8,8,1.0,0.873275,-0.062256,0.633499,-0.169531,0.452925,-0.174406,0.214923,-0.244422,...,-0.288446,-0.121617,-0.279057,-0.252523,-0.116271,-0.382603,-0.025546,-0.345149,0.089970,1.960940
9,9,1.0,0.700726,0.162233,0.170163,0.112041,-0.066100,0.033766,0.005732,-0.034823,...,-0.194770,-0.550983,-0.260088,-0.808985,-0.306165,-0.436931,-0.340459,-0.093636,-0.141741,5.800941


## Trotter simulation (Simulator, Noisy)

In [12]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]
    heisenberg_sim_noisy = HeisenbergModel(
        n_qubits, graphs[i], backend=backend_sim_noisy
    )

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=0
            )
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=1
            )
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=2
            )
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=3
            )
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/56


Creating circuits for sample 1/56
Creating circuits for sample 2/56
Creating circuits for sample 3/56
Creating circuits for sample 4/56
Creating circuits for sample 5/56
Creating circuits for sample 6/56
Creating circuits for sample 7/56
Creating circuits for sample 8/56
Creating circuits for sample 9/56
Creating circuits for sample 10/56
Creating circuits for sample 11/56
Creating circuits for sample 12/56
Creating circuits for sample 13/56
Creating circuits for sample 14/56
Creating circuits for sample 15/56
Creating circuits for sample 16/56
Creating circuits for sample 17/56
Creating circuits for sample 18/56
Creating circuits for sample 19/56
Creating circuits for sample 20/56
Creating circuits for sample 21/56
Creating circuits for sample 22/56
Creating circuits for sample 23/56
Creating circuits for sample 24/56
Creating circuits for sample 25/56
Creating circuits for sample 26/56
Creating circuits for sample 27/56
Creating circuits for sample 28/56
Creating circuits for sample 

In [13]:
# Run jobs in batch
jobs = []  # For AerSimulator, we can't use job ids.
with Batch(backend=backend_sim_noisy):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/56
Running circuits for sample 1/56
Running circuits for sample 2/56
Running circuits for sample 3/56
Running circuits for sample 4/56
Running circuits for sample 5/56
Running circuits for sample 6/56
Running circuits for sample 7/56
Running circuits for sample 8/56
Running circuits for sample 9/56
Running circuits for sample 10/56
Running circuits for sample 11/56
Running circuits for sample 12/56
Running circuits for sample 13/56
Running circuits for sample 14/56
Running circuits for sample 15/56
Running circuits for sample 16/56
Running circuits for sample 17/56
Running circuits for sample 18/56
Running circuits for sample 19/56
Running circuits for sample 20/56
Running circuits for sample 21/56
Running circuits for sample 22/56
Running circuits for sample 23/56
Running circuits for sample 24/56
Running circuits for sample 25/56
Running circuits for sample 26/56
Running circuits for sample 27/56
Running circuits for sample 28/56
Running circuits for sam

In [14]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,936fa631-9e7f-44a2-b1f2-56ed5e36f890,0,DONE
1,38ab8908-4c16-4aff-b7a5-df4a1b84bc23,1,DONE
2,b02e5252-7d44-46fb-b028-8080d8489c9a,2,DONE
3,8e24da51-5ff9-4b1a-9fcc-096c50470180,3,DONE
4,a79f6373-6266-4c1c-abd5-78b971fbe78d,4,DONE
5,dd8a06e0-357b-48a1-99a7-2409ecd4b4e3,5,DONE
6,0974a91d-a0bd-4454-a867-396731501729,6,DONE
7,4a6bb8ca-ac65-4a4b-92a5-bb32cc04eeb4,7,DONE
8,3e8e1cee-8ab8-4f48-88ef-18539af51bb7,8,DONE
9,a87291da-cdb5-469f-ac8d-aef8ce053d23,9,DONE


In [15]:
# Post-process
data_sim_noisy = []
probs_phase0_sim_noisy = {}
probs_phase1_sim_noisy = {}
probs_phase2_sim_noisy = {}
probs_phase3_sim_noisy = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim_noisy[f"sample{i}"] = {}
    probs_phase1_sim_noisy[f"sample{i}"] = {}
    probs_phase2_sim_noisy[f"sample{i}"] = {}
    probs_phase3_sim_noisy[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )
        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim_noisy.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_noisy_df = pd.DataFrame(data_sim_noisy, columns=columns)
display(data_sim_noisy_df)

# Save the simulation data
data_sim_noisy_df.to_json(f"{path}/data_sim_noisy_df.json")
with open(f"{path}/probs_phase0_sim_noisy.json", "w") as f:
    json.dump(probs_phase0_sim_noisy, f)

with open(f"{path}/probs_phase1_sim_noisy.json", "w") as f:
    json.dump(probs_phase1_sim_noisy, f)

with open(f"{path}/probs_phase2_sim_noisy.json", "w") as f:
    json.dump(probs_phase2_sim_noisy, f)

with open(f"{path}/probs_phase3_sim_noisy.json", "w") as f:
    json.dump(probs_phase3_sim_noisy, f)

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,0.906250,0.659220,0.317255,0.176135,0.369393,-0.027201,0.141089,-0.061191,-0.113881,...,-0.184595,0.122709,-0.104598,0.018480,0.036941,0.004582,0.023821,-0.024437,0.006161,7.391106
1,1,0.918945,0.438658,-0.205966,0.030005,-0.091615,-0.005609,0.179611,0.143433,0.275466,...,0.085516,-0.131159,0.021275,-0.142401,-0.070570,0.042149,0.071163,-0.009759,0.100154,7.380861
2,2,0.906250,0.697359,-0.173881,0.307337,-0.167424,-0.065841,-0.028827,-0.216285,0.180567,...,0.279653,-0.113195,0.303567,0.023757,0.235617,0.038129,0.114373,-0.035167,0.086663,1.560675
3,3,0.921875,0.603148,0.153091,0.187414,0.109748,0.037666,0.051465,0.018588,-0.012759,...,0.173945,0.055681,0.343953,0.147960,0.390451,-0.180156,0.302604,-0.124250,0.167141,5.663484
4,4,0.923828,0.579428,-0.181111,0.107323,-0.089270,-0.001344,-0.027118,0.005616,0.049679,...,0.294675,0.223878,0.305084,0.323179,0.159171,0.189229,-0.035921,0.039666,-0.002614,1.889904
5,5,0.916992,0.757055,-0.109238,0.607726,-0.186466,0.434696,-0.170520,0.329327,-0.048752,...,-0.042014,0.178898,0.062301,0.060905,0.154961,-0.007589,0.138485,-0.076097,0.188992,2.056023
6,6,0.913086,0.653140,-0.185262,0.332277,-0.159419,0.061951,-0.061218,-0.060076,0.023430,...,0.120583,0.232308,0.153032,0.393633,-0.068718,0.384430,-0.269719,0.232155,-0.434077,1.492621
7,7,0.900391,0.473417,-0.654447,-0.153164,-0.649445,-0.602104,-0.056594,-0.372146,0.354158,...,0.375436,0.378732,0.013896,0.226008,-0.243170,-0.181638,-0.308831,-0.275687,-0.072091,0.238580
8,8,0.890625,0.717717,-0.051726,0.524237,-0.109875,0.358785,-0.111745,0.198513,-0.189339,...,-0.203282,-0.095755,-0.196895,-0.202072,-0.083487,-0.276771,-0.029970,-0.259274,0.066303,1.960940
9,9,0.907227,0.578506,0.145132,0.150406,0.087341,-0.034555,0.011635,0.053442,-0.060628,...,-0.113998,-0.414247,-0.175250,-0.589162,-0.225187,-0.303916,-0.240255,-0.068653,-0.071168,5.800941
